### Data Quality Analysis  
##### All features are wrapped in python functions (use Run All to define all functions and launch them individually)

In [52]:
import os
import sqlite3
import pandas as pd
import pandas_datareader as pdr
import numpy as np
import datetime
import plotly.graph_objects as go
import talib as ta 


#### RUN this feature only once if you do NOT have historical data  
All price data will be stored in a CSV file under (CSV folder)

In [2]:
# Function to get historical data from Yahoo finance and store as a CSV file
# Also cumulates symbols that are not available on Yahoo Finance for further investigation
# TODO: Modify this function to pass start and end date for extraction if data before 2014 is required
def yahoo_to_csv(ticker, exchange):
    start_date = "2014-01-01"
    end_date   = str(datetime.datetime.now().strftime("%Y-%m-%d"))
    if exchange == "tsx":
        yahoo_symbol = ticker.replace(".", "-") + ".TO"
    else:
        yahoo_symbol = ticker.replace(".", "-") + ".V"

    try:
        data = pdr.DataReader(yahoo_symbol, "yahoo", start_date, end_date)
        data["Ticker"] = ticker
        data.index = pd.to_datetime(data.index)
        data.to_csv(f"CSV/{ticker}.csv", index_label="Date", mode="w", date_format="%Y-%m-%d %H:%M:%S")
        return data
    except Exception as e:
        print(f"Unable to read Data from Yahoo : {e}")
        return None

# yahoo_to_csv("SHOP")

##### Feature to extract history price data from TSX

In [3]:
# Web scraper for TSX (when we can't get it from yahoo finance)
import time
import random
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [4]:
def open_browser():
    # Setup Selenium browser
    CHROME_DRIVER_LOCATION = "chromedriver.exe"
    service_object = Service(CHROME_DRIVER_LOCATION)

    OPTIONS = webdriver.ChromeOptions()
    OPTIONS.add_argument('--ignore-certicate-errors')
    OPTIONS.add_argument('--incognito')
    #OPTIONS.add_argument('--headless')
    OPTIONS.add_experimental_option('excludeSwitches', ['enable-logging'])
    
    #self.driver = webdriver.Chrome(executable_path=CHROME_DRIVER_LOCATION,options=OPTIONS)
    driver = webdriver.Chrome(service=service_object, options=OPTIONS)
    WebDriverWait(driver, 10)

    return driver

In [5]:
def close_add(driver):
    try:
        close_ad_btn = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'ssrt-close-anchor-button')))
    except Exception as e:
        print(f"Unable to close_ads ")  
        return False 

    try:
        close_ad_btn.click()
        return True
    except:
        return False

In [6]:
def next_page(driver):
    # btn_next = driver.find_element(By.XPATH, "//button[@data-testid='next-button']")
    try:
        next_btn = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//button[@data-testid='next-button']")))
        next_btn.click()
        #print(f"Next button element found on page...")
        return True
    except Exception as e:
        print(f"Next button element NOT FOUND :")  
        return False 

In [7]:
def tsx_to_csv(driver, ticker, num_pages):
    url = f"https://money.tmx.com/en/quote/{ticker}/trade-history?selectedTab=price-history"
    driver.get(url)
    random_delay = random.randint(2, 6)
    #print(f"Random sleep delay : {random_delay}")
    time.sleep(random_delay)
    ad_closed = close_add(driver)

    column_names = ["Date", "Open ($)", "High ($)", "Low ($)", "Close ($)", "VWAP ($)", "Change ($)", "Change (%)", "Volume", "Trade Value", "# Trades"]
    data_df = pd.DataFrame(columns=column_names)

    if ad_closed:
        # Loop through pages
        for i in range(0, num_pages):
            html_page = driver.page_source 
            try:
                html_data = pd.read_html(html_page)
            except:
                html_data = []
            
            if len(html_data) == 0:
                i = num_pages
                prices_df = None
            else:
                prices_df = html_data[0]
                if prices_df.empty:
                    i = num_pages
                else:
                    data_df = data_df.append(prices_df)
                    prices_df = None
                    # click next page button
                    next_page(driver)
                    random_delay = random.randint(2, 6)
                    #print(f"Random sleep delay NEXT PAGE : {random_delay}")
                    time.sleep(random_delay)

        if not data_df.empty:
            data_df.index = pd.to_datetime(data_df["Date"], infer_datetime_format=True)
            data_df.drop(columns=["Date","VWAP ($)","Change ($)","Change (%)","Trade Value","# Trades"], inplace=True)
            data_df = data_df.rename(columns={'Open ($)': 'Open', 'High ($)': 'High', 'Low ($)': 'Low', 'Close ($)': 'Close'})
            data_df["Adj Close"] = -1
            data_df["Ticker"] = ticker
            data_df.to_csv(f"TSXCSV/{ticker}.csv", index_label="Date", mode="w", date_format="%Y-%m-%d %H:%M:%S")
            return data_df
        else:
            print(f"No data scrapped - no prices found, {ticker}")
            return None
    else:
        print(f"No data scrapped - problem with ads, {ticker}")
        return None

# driver = open_browser()
# p1 = tsx_to_csv(driver, "RY.PR.M", 100)

In [ ]:
def loop_through_missing_symbols(amount):
    notfound_symbols = pd.read_csv("notfound.csv", header=None)
    symbols_list = notfound_symbols[0].tolist()
    start_index = 0
    step = amount
    driver = open_browser()
    for symbol in symbols_list[start_index:start_index+step]:
        p1 = tsx_to_csv(driver, symbol, 100)
        print(f"Symbol : {symbol}")
    
# loop_through_missing_symbols(50)


In [50]:
# Export SQLite3 Prices to CSV file for github push using yearly export
def yearly_prices_to_csv(year):
    #year="2022"
    conn = sqlite3.connect("TSX_Quality.sqlite")
    sql = f"SELECT * FROM 'prices_daily' WHERE Date LIKE '{year}%' ORDER BY ticker ASC, Date DESC"
    data = pd.read_sql_query(sql, conn)
    #data.drop(labels="index", axis=1, inplace=True)
    #data["Date"] = pd.to_datetime(data["Date"], infer_datetime_format=True)
    #data["Date"] = data["Date"].dt.date
    data.to_csv(f"prices_{year}.csv", index=False)

#yearly_prices_to_csv("2013")

In [11]:
# Function to return a Dataframe with all symbols available in database
def get_all_symbols():
    """ Get all symbols in database as a DataFrame """
    conn = sqlite3.connect("TSX_Prices.sqlite")
    sql = f"SELECT * FROM symbols ORDER BY ticker"
    data = pd.read_sql_query(sql, conn, index_col="ticker")
    data.drop(labels=["index", "url", "yahoo"], axis=1, inplace=True )

    return data

In [32]:
# Heavy function to retreive all prices from database (more than 5 million rows, takes many seconds to execute)
def get_all_prices():
    conn = sqlite3.connect("TSX_Quality.sqlite")
    sql = f"SELECT * FROM prices_daily ORDER BY UPPER(Ticker) ASC, Date ASC"
    prices = pd.read_sql_query(sql, conn, index_col="Date")
    prices.index = pd.to_datetime(prices.index)
    #prices = pd.read_sql_query(sql, conn, index_col="ticker")
    #prices.drop("index", axis=1, inplace=True)
    
    return prices


In [ ]:
def csv_to_dataframe(folder, file):
    data = pd.read_csv(f"{folder}/{file}", index_col="Date")
    data.index = pd.to_datetime(data.index)
    data = data[["Ticker","Open","High","Low","Close","Volume"]]
    return data

def insert_new_prices_in_DB(new_prices, db_prices, conn):
    ticker = new_prices["Ticker"].values[0]
    existing_prices = db_prices.loc[db_prices["Ticker"] == ticker]
    filter = new_prices.index.isin(existing_prices.index)
    new_prices.drop(new_prices[filter].index, inplace = True)
    #conn = sqlite3.connect("TSX_Quality.sqlite")    
    new_prices.to_sql("Prices_Daily", conn, if_exists='append', index=True)
    return new_prices

def loop_through_local_csv(existing_prices, conn, folder):
    csv_files = os.listdir(f"{folder}")
    for file in csv_files:
        new_data = csv_to_dataframe(folder, file)
        insert_new_prices_in_DB(new_data, existing_prices, conn)
        print(f"Finished processing : {file}")

# conn1 = sqlite3.connect("TSX_Quality.sqlite")
# sql = f"SELECT * FROM prices_daily ORDER BY Ticker ASC, Date ASC"
# db_prices = pd.read_sql_query(sql, conn1, index_col="Date")
# loop_through_local_csv(db_prices, conn1, "CSV")

conn1 = sqlite3.connect("TSX_Quality.sqlite")
db_prices = get_all_prices()
loop_through_local_csv(db_prices, conn1, "TSXCSV")

In [13]:
# Cell to loop through all symbols and extract price data from yahoo
def loop_through_symbols_on_yahoo():
    not_found_on_yahoo = []
    df = get_all_symbols()
    df["YahooExists"] = False
    #df_t = df[40:70]
    df_t = df

    for index, row in df_t.iterrows():
        symbol = index
        exchange = row["exchange"]
        result = yahoo_to_csv(symbol, exchange)
        if result is not None:
            df.at[symbol, "YahooExists"] = True
        else:
            not_found_on_yahoo.append(symbol)

    print(not_found_on_yahoo)
    notfound_df = pd.DataFrame(not_found_on_yahoo)
    notfound_df.to_csv("notfoundonyahoo.csv", mode="w", index=False, header=False )
# 3634 elements from yahoo in 1:11 hours

#loop_through_symbols_on_yahoo()

### DATA QUALITY INDICATORS

In [37]:
# Identify the first price data for every ticker (to help in finding previous date for GAP analysis)
# Prices must be sorted by ascending  ticker symbol and ascending dates
# For every ticker+date combination, insert the date of the previous price data fo rthe same ticker (to calculate the number of days between data and detect missing prices)
def detect_missing_prices(prices):
    prices["new_ticker"] = np.where(prices["Ticker"] != prices["Ticker"].shift(1), "New", "")
    prices["cur_date"]  = pd.to_datetime(prices.index, format="%Y-%m-%d", errors='coerce')
    prices["prev_date"] = pd.to_datetime(np.where(prices["new_ticker"] != "New", prices["cur_date"].shift(1), None), format="%Y-%m-%d", errors='coerce')

    # Calculate date gaps in prices using succesive dates for tickers in database
    prices["GAP"] = prices["cur_date"] - prices["prev_date"]
    prices["missing"] = prices["GAP"] > datetime.timedelta(days=5)

    
# Show date GAPS for a specific symbol
def show_missing_prices(prices, ticker=None):
    if ticker is None:
        filter = (prices["missing"] == True)
    else:
        filter = (prices["missing"] == True) & (prices["Ticker"] == ticker)
    
    missing_data = prices.loc[filter]
    return missing_data

prices = get_all_prices()
detect_missing_prices(prices)
missing_df = show_missing_prices(prices)


In [35]:
#missing_df
missing_df.sort_values(by='GAP', ascending=False)
#missing_df.sort_values(by='GAP', ascending=False).head(10)

,Ticker,Open,High,Low,Close,Volume,new_ticker,cur_date,prev_date,GAP,missing
Date,,,,,,,,,,,
2020-06-01,SBC.PR.A,9.74,9.81,9.74,9.75,24000.0,,2020-06-01,2019-11-22,192 days,True
2016-09-20,WN.PR.C,25.0,25.03,25.0,25.03,800.0,,2016-09-20,2016-05-06,137 days,True
2016-05-09,SBC.PR.A,10.07,10.07,10.07,10.07,100.0,,2016-05-09,2015-12-24,137 days,True
2019-02-28,UNC.PR.B,-,-,-,24.333,0.0,,2019-02-28,2018-11-09,111 days,True
2015-07-16,RY.PR.J,24.32,24.51,24.32,24.51,6840.0,,2015-07-16,2015-03-27,111 days,True
...,...,...,...,...,...,...,...,...,...,...,...
2015-03-31,HUL.U,9.34070014953613,9.34070014953613,9.34070014953613,9.34070014953613,0.0,,2015-03-31,2015-03-25,6 days,True
2015-07-30,HTB,51.2299995422363,51.2299995422363,51.1399993896484,51.1399993896484,0.0,,2015-07-30,2015-07-24,6 days,True
2015-06-09,HTB,47.75,47.75,47.5099983215332,47.6100006103516,500.0,,2015-06-09,2015-06-03,6 days,True


In [16]:
#filter = prices["new_ticker"] == "New"
#prices.loc[filter]
# df["Trend"] = np.where(df["Close"] > df["SMA200"], "Up", "Down")